In [16]:
import ipywidgets as widgets
from IPython.display import display, HTML
import os
import io
import time
import csv
import base64
from pathlib import Path
from fastai.torch_basics import *
from fastai.text.all import *
from fastai.text.core import *
from fastai import *
from fastai.text.data import *
import pandas as pd


In [17]:
path = Path().absolute()

In [18]:
classifier = load_learner(path/'models/classificator.pkl')

In [19]:
print("Select file...")
btn_upload = widgets.FileUpload(accept=".csv")
btn_upload

Select file...


FileUpload(value={}, accept='.csv', description='Upload')

In [20]:
out = widgets.Output()
@out.capture()
def on_upload(event):

    content = list(btn_upload.value.values())[event['new'] - 1]['content']
    content = io.StringIO(content.decode('utf-8'))
    
    data = pd.read_csv(content, sep = ';', header=None)
    data.columns = ["text"]
    
    print ("Computing...\n")
    
    with classifier.no_bar():
        preds = [classifier.predict(string)[0] for string in data['text']]
    
    data['label'] = preds
    
    display(create_download_link(data))



In [21]:
def create_download_link( df, title = "Download results ", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [22]:
btn_upload.observe(on_upload, "_counter")

In [ ]:
out